# **Primer análisis de datos extraidos**

#### Incluimos las librerías necesarias para analizar los datos

In [13]:
import pandas as pd
import json
from collections import defaultdict
import re
from datetime import datetime, timedelta

#### Leemos los datos del JSON y lo guardamos en la variable *datos*

In [14]:
datos = pd.read_json('traces_Articoding_Escolapias.json')

# Comprobamos si el objeto que miramos es un nivel

In [113]:
def get_tipo(tipo):
    return tipo.rsplit('/',1)[-1]

def es_nivel(url):
    subtipo = get_tipo(url)
    return subtipo == 'level'


#EJ:
prueba = datos.head(5) 

for caso in prueba.to_numpy():
    tipo = caso[4]['definition']['type']
    nivel = es_nivel(tipo)
    print("->")
    print(caso[4]['id'])
    if nivel:
        print(get_tipo(caso[4]['definition']['type'])+"\nEs un nivel")
    else:
        print(get_tipo(caso[4]['definition']['type'])+"\nNo es nivel")


->
https://simva-api.simva.e-ucm.es/activities/628179b5a6ba41003d3bada2/tip_0_bienvenido
dialog-fragment
No es nivel
->
https://simva-api.simva.e-ucm.es/activities/628179b5a6ba41003d3bada2/articoding
serious-game
No es nivel
->
https://simva-api.simva.e-ucm.es/activities/628179b5a6ba41003d3bada2/articoding
serious-game
No es nivel
->
https://simva-api.simva.e-ucm.es/activities/628179b5a6ba41003d3bada2/categories_main
screen
No es nivel
->
https://simva-api.simva.e-ucm.es/activities/628179b5a6ba41003d3bada2/tutorials_general
screen
No es nivel


## Obtenemos el codigo de inicio y fin de nivel

In [116]:

prueba = datos.head(20)

tiempos = set()   
#Buscamos el nombre de los tiempos de inicio y fin, los cuales se encuentran en 'verb'
for time in prueba['verb']:
    tiempos.add(time['id'])
#Guardamos el link de inicio y fin
print(tiempos)
initialized = "http://adlnet.gov/expapi/verbs/initialized"
completed = "http://adlnet.gov/expapi/verbs/completed"
print('Initialized: '+ initialized)
print('Completed: '+ completed)


{'http://adlnet.gov/expapi/verbs/progressed', 'https://w3id.org/xapi/seriousgames/verbs/accessed', 'http://adlnet.gov/expapi/verbs/initialized', 'http://adlnet.gov/expapi/verbs/completed'}
Initialized: http://adlnet.gov/expapi/verbs/initialized
Completed: http://adlnet.gov/expapi/verbs/completed


#### Lista con los tiempos de cada usuario en cada nivel

Guarda los tiempos que ha tardado cada usuario en terminar los cada nivel, en caso de completarlo varias veces, guarda todos los tiempos

In [144]:
def seconds_to_hours(time):
    print(datetime.timedelta(time))

def get_completion_time_list(datos):
    #verb nos dice si se ha iniciado o completado un nivel
    #Buscamos guardar en un diccionario de usuarios diccionarios de niveles con los tiempos de ejecución
    userList = defaultdict(lambda: defaultdict(list))
    #ketTuple nos permite saber si el usuario ya ha jugado ese nivel con anterioridad
    iniTime = defaultdict()
    tam = len(datos)
    for elemento in datos.to_numpy():
        #Comprobamos que el objeto sea un nivel
        if get_tipo(elemento[4]['definition']['type']) == 'level':
            verb = elemento[2]['id']
            name = elemento[0]['name']
            timestamp = elemento[5]
            level = elemento[4]['id']

            #Si es un nuevo nivel marcamos el tiempo de inicio
            if (name,level) not in iniTime or verb == initialized:
                iniTime[(name,level)] = timestamp
            else:
                userList[name][level].append(timestamp-iniTime[(name,level)]) 
                iniTime.pop((name,level))

    return userList


            
time_list = get_completion_time_list(datos)
#Mostramos los tiempos que tiene el usuario 'gqoj' cada vez que completó el primer tutorial
for element in time_list['gqoj']["https://simva-api.simva.e-ucm.es/activities/628179b5a6ba41003d3bada2/tutorials_1"]:
    print(str(element))


0 days 00:00:16.992000
0 days 00:00:48.227000
0 days 00:00:32.308000


## Calculamos el tiempo de cada nivel

In [34]:
def mostrar_tiempo(segundos):
    horas = str(segundos // 3600)
    minutos = str((segundos % 3600) // 60)
    segundos = str(segundos % 60)
    return str(horas+":"+minutos+":"+segundos)


#Calculamos las diferencias de todos los tiempos y sacamos datos de tiempos incompletos
userTimes = defaultdict(defaultdict)
for user in userList:
    for level in userList[user]:
        time = userList[name][level][completed] - userList[name][level][initialized]
        userTimes[name][level] = time.seconds
        print(mostrar_tiempo(userTimes[name][level]))


userTimes



0:0:7


## Calculamos el tiempo medio de cada nivel

* Lo primero sería extraer su primer *timestamp*  

In [6]:
inicio = ""
for i in datos.to_numpy():
    if(i[0]["name"] == "gqoj"):
        inicio = i[5]
        break

In [7]:
inicio

Timestamp('2022-05-16 10:36:35.361000+0000', tz='UTC')

* Ahora que ya tenemos el inicio de la partida vamos a buscar el fin de la partida, para ello nos recorremos todo el JSON y nos quedamos con el último *timestamp* del jugador

In [8]:
fin = ""
for i in datos.to_numpy():
    if(i[0]["name"] == "gqoj"):
        fin = i[5]

In [9]:
fin

Timestamp('2022-05-23 16:01:55.895000+0000', tz='UTC')

Parece que las trazas no son todas del mismo día, hay que buscar la forma de **filtrar por días**.

#### Ahora que ya sabemos extraer el *timestamp* de un jugador en concreto vamos a intentar recoger los de todos los jugadores

* Guardaremos los datos en un diccionario cuya clave será el **id** del jugador y su valor una pareja con el *timestamp* de inicio y de fin

In [10]:
tiempos = dict()
for i in datos.to_numpy():
    idUser = i[0]["name"]
    timeStamp = i[5]
    
    if(idUser in tiempos):
        timeStampIni = tiempos[idUser][0]
        tiempos[idUser] = (timeStampIni, timeStamp)
    else:
        tiempos[idUser] = (timeStamp, "")

In [11]:
pd.DataFrame(tiempos)

,gqoj,hgfx,gton,uala,qeqyj,fkmtp,hjpbw,bdwyj,eprdu,ftpnb,...,bajbm,eayrh,orjyn,prpej,fmupf,vkfnt,xbmmw,pleui,olugf,epjcx
0,2022-05-16 10:36:35.361000+00:00,2022-05-16 10:48:06.634000+00:00,2022-05-16 22:12:06.760000+00:00,2022-05-17 07:39:39.267000+00:00,2022-05-18 07:10:24.601000+00:00,2022-05-18 07:10:44.894000+00:00,2022-05-18 07:10:59.919000+00:00,2022-05-18 07:11:02.294000+00:00,2022-05-18 07:11:01.092000+00:00,2022-05-18 07:11:18.662000+00:00,...,2022-05-24 09:33:01.510000+00:00,2022-05-24 09:33:02.422000+00:00,2022-05-24 09:33:07.855000+00:00,2022-05-24 09:33:41.060000+00:00,2022-05-24 09:34:20.130000+00:00,2022-05-24 09:34:23.401000+00:00,2022-05-24 09:34:22.295000+00:00,2022-05-24 09:34:32.727000+00:00,2022-05-24 09:36:36.250000+00:00,2022-05-24 12:20:00.205000+00:00
1,2022-05-23 16:01:55.895000+00:00,2022-05-16 10:54:01.016000+00:00,2022-05-16 22:12:21.778000+00:00,2022-05-17 07:42:45.832000+00:00,2022-05-18 10:10:50.954000+00:00,2022-05-18 10:10:45.117000+00:00,2022-05-18 10:10:58.339000+00:00,2022-05-18 10:13:10.064000+00:00,2022-05-18 10:10:53.289000+00:00,2022-05-18 10:10:55.723000+00:00,...,2022-05-24 12:13:54.834000+00:00,2022-05-24 12:12:45.295000+00:00,2022-05-24 12:14:02.923000+00:00,2022-05-24 12:09:03.889000+00:00,2022-05-24 12:11:58.891000+00:00,2022-05-24 12:13:18.012000+00:00,2022-05-24 12:12:04.445000+00:00,2022-05-24 12:12:17.773000+00:00,2022-05-24 12:09:25.385000+00:00,2022-05-24 12:58:14.766000+00:00


* Parece que ya tenemos los datos que queríamos de cada jugador, asumimos que son los correctos porque los tiempos del jugador *gqoj* coinciden con los que hemos sacado antes

* Ahora toca hacer la resta para ver el tiempo de juego de los jugadores 